In [38]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt # No longer needed
# from collections import OrderedDict # No longer needed for legend
# import matplotlib.ticker as mtick # No longer needed

# --- Plotly Imports ---
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly.express as px # For color palettes

# --- Настройки ---
days_history = 180
today = pd.Timestamp('2025-04-19')
# today = None # Для использования последней даты из данных

# --- Шаг 1: Загрузка данных о портфеле ---
portfolio_data = {
    "ID": [2, 0, 1, 3], # ID могут быть произвольными
    "Дата": ["2025-02-09T14:21:24.000", "2025-04-01T14:21:01.000", "2025-03-05T14:21:17.000", "2025-01-12T14:29:48.000"], # УКАЖИТЕ ВАШИ РЕАЛЬНЫЕ ДАТЫ ПОКУПОК!
    "Актив": ["LTCUSDT", "BNBUSDT", "BTCUSDT", "HBARUSDT"],
    "Общая стоимость": [20.00, 1000.00, 1000.00, 1000.00] # КЛЮЧЕВОЙ ПАРАМЕТР
    # Остальные столбцы из исходных данных портфеля не нужны для этого расчета
}
portfolio_df = pd.DataFrame(portfolio_data)
portfolio_df['Дата'] = pd.to_datetime(portfolio_df['Дата'])
portfolio_df = portfolio_df.sort_values(by='Дата').reset_index(drop=True)

# --- Шаг 2: Загрузка исторических данных ---
# Assume files are in the correct location as per the original script
try:
    # Укажите ПРАВИЛЬНЫЕ пути
    btc_data = pd.read_csv('D:\\__projects__\\diploma\\portfolios-optimization\\data\\BTCUSDT_hourly_data.csv')
    bnb_data = pd.read_csv('D:\\__projects__\\diploma\\portfolios-optimization\\data\\BNBUSDT_hourly_data.csv')
    ltc_data = pd.read_csv('D:\\__projects__\\diploma\\portfolios-optimization\\data\\LTCUSDT_hourly_data.csv')
    hbar_data = pd.read_csv('D:\\__projects__\\diploma\\portfolios-optimization\\data\\HBARUSDT_hourly_data.csv')
except FileNotFoundError as e:
    print(f"Ошибка: Файл не найден. Проверьте путь: {e}")
    exit()

def preprocess_data(df, asset_name):
    df['Open time'] = pd.to_datetime(df['Open time'])
    df = df.set_index('Open time')
    df = df[['Close']].rename(columns={'Close': f'{asset_name}_Price'})
    df[f'{asset_name}_Price'] = df[f'{asset_name}_Price'].astype(float)
    return df

btc_data = preprocess_data(btc_data, 'BTCUSDT')
bnb_data = preprocess_data(bnb_data, 'BNBUSDT')
ltc_data = preprocess_data(ltc_data, 'LTCUSDT')
hbar_data = preprocess_data(hbar_data, 'HBARUSDT')

historical_prices = pd.concat([btc_data, bnb_data, ltc_data, hbar_data], axis=1)

if today is None:
    today = historical_prices.index.max()
# Ensure today is timezone-naive or consistent with index
if historical_prices.index.tz is not None:
    today = today.tz_localize(None) # Make today timezone naive if index is naive
else:
    today = today # Assume today is already naive if index is naive

start_date_history = today - pd.Timedelta(days=days_history)

# --- Шаг 3: Фильтрация, подготовка и расчеты ---

# Ensure start_date_history is compatible with index timezone (or lack thereof)
if historical_prices.index.tz is not None and start_date_history.tz is None:
     start_date_history = start_date_history.tz_localize(historical_prices.index.tz)
elif historical_prices.index.tz is None and start_date_history.tz is not None:
     start_date_history = start_date_history.tz_localize(None)


historical_prices_filtered = historical_prices[(historical_prices.index >= start_date_history) & (historical_prices.index <= today)].copy()
historical_prices_filtered = historical_prices_filtered.ffill().bfill()

if historical_prices_filtered.isnull().values.any():
    print("Предупреждение: NaN в данных после заполнения. Удаление строк.")
    historical_prices_filtered = historical_prices_filtered.dropna()

portfolio_df['Purchase_Price_Actual'] = np.nan
portfolio_df['Actual_Purchase_Time_Index'] = pd.NaT

# Ensure portfolio 'Дата' is timezone-naive or consistent with index
if historical_prices_filtered.index.tz is not None:
    portfolio_df['Дата'] = portfolio_df['Дата'].dt.tz_localize(None).dt.tz_localize(historical_prices_filtered.index.tz)
else:
    portfolio_df['Дата'] = portfolio_df['Дата'].dt.tz_localize(None) # Make naive

print("Поиск цен на момент покупки...")
for index, row in portfolio_df.iterrows():
    asset = row['Актив']
    purchase_date = row['Дата']
    price_col = f'{asset}_Price'
    if price_col not in historical_prices_filtered.columns: continue

    # Find the first available time >= purchase time in the filtered data index
    relevant_prices_index = historical_prices_filtered.index[historical_prices_filtered.index >= purchase_date]

    if not relevant_prices_index.empty:
        actual_purchase_time_index = relevant_prices_index[0]
        try:
            purchase_price = historical_prices_filtered.loc[actual_purchase_time_index, price_col]
            if pd.notna(purchase_price) and purchase_price > 0:
                portfolio_df.loc[index, 'Purchase_Price_Actual'] = purchase_price
                portfolio_df.loc[index, 'Actual_Purchase_Time_Index'] = actual_purchase_time_index
                # print(f"  Found price for {asset} at {actual_purchase_time_index}: {purchase_price}") # Debug print
            else:
                 print(f"Предупреждение: Не найдена или некорректная цена для {asset} в {actual_purchase_time_index}. Цена: {purchase_price}")
                 portfolio_df.loc[index, 'Actual_Purchase_Time_Index'] = pd.NaT # Помечаем как невалидное время
        except KeyError:
             print(f"Предупреждение: KeyError при поиске цены для {asset} в {actual_purchase_time_index}.")
             portfolio_df.loc[index, 'Actual_Purchase_Time_Index'] = pd.NaT # Помечаем как невалидное время
    else:
        # If no index found >= purchase date, try the closest *before* if it's within the filtered range
        relevant_prices_index_before = historical_prices_filtered.index[historical_prices_filtered.index < purchase_date]
        if not relevant_prices_index_before.empty:
            actual_purchase_time_index = relevant_prices_index_before[-1] # Closest before
            print(f"Предупреждение: Нет данных для {asset} в или после {purchase_date}. Используется ближайшее время *до*: {actual_purchase_time_index}")
            try:
                 purchase_price = historical_prices_filtered.loc[actual_purchase_time_index, price_col]
                 if pd.notna(purchase_price) and purchase_price > 0:
                     portfolio_df.loc[index, 'Purchase_Price_Actual'] = purchase_price
                     portfolio_df.loc[index, 'Actual_Purchase_Time_Index'] = actual_purchase_time_index
                 else:
                    print(f"Предупреждение: Некорректная цена для {asset} в {actual_purchase_time_index} (время до покупки). Цена: {purchase_price}")
                    portfolio_df.loc[index, 'Actual_Purchase_Time_Index'] = pd.NaT
            except KeyError:
                print(f"Предупреждение: KeyError при поиске цены для {asset} в {actual_purchase_time_index} (время до покупки).")
                portfolio_df.loc[index, 'Actual_Purchase_Time_Index'] = pd.NaT
        else:
            print(f"Предупреждение: Нет исторических данных для {asset} около {purchase_date} в отфильтрованном диапазоне.")
            portfolio_df.loc[index, 'Actual_Purchase_Time_Index'] = pd.NaT # Помечаем как невалидное время


portfolio_df.dropna(subset=['Actual_Purchase_Time_Index', 'Purchase_Price_Actual'], inplace=True)
portfolio_df = portfolio_df.reset_index(drop=True) # Переиндексация после dropna

if len(portfolio_df) == 0:
    print("Ошибка: Нет действительных покупок для анализа в указанном диапазоне дат.")
    exit()

# Расчет кумулятивной стоимости (бенчмарк)
print("Расчет кумулятивной стоимости...")
historical_prices_filtered['Cumulative_Cost'] = 0.0
for _, row in portfolio_df.iterrows():
    cost = row['Общая стоимость']
    purchase_time = row['Actual_Purchase_Time_Index']
    historical_prices_filtered.loc[historical_prices_filtered.index >= purchase_time, 'Cumulative_Cost'] += cost

# Расчет стоимости, P&L и процентного вклада для КАЖДОЙ отдельной покупки
print("Расчет вклада, P&L и % вклада для каждой покупки...")
purchase_value_cols = []
purchase_pnl_cols = []
purchase_perc_contrib_cols = []
purchase_labels = [] # Метки для легенды и идентификации
purchase_ids_assets = [] # Для сопоставления с цветами

for index, purchase_row in portfolio_df.iterrows():
    purchase_id = purchase_row['ID']
    asset = purchase_row['Актив']
    initial_investment = purchase_row['Общая стоимость']
    purchase_price = purchase_row['Purchase_Price_Actual']
    purchase_time = purchase_row['Actual_Purchase_Time_Index']
    price_col = f'{asset}_Price'

    if price_col not in historical_prices_filtered.columns:
        print(f"Предупреждение: Колонка цен {price_col} отсутствует. Пропуск покупки ID {purchase_id}.")
        continue

    value_col_name = f"Value_ID{purchase_id}_{asset}"
    pnl_col_name = f"PnL_ID{purchase_id}_{asset}"
    perc_contrib_col_name = f"PercContrib_ID{purchase_id}_{asset}"
    label = f"{asset} (ID:{purchase_id}, ${initial_investment:.2f})" # Метка для легенды

    purchase_value_cols.append(value_col_name)
    purchase_pnl_cols.append(pnl_col_name)
    purchase_perc_contrib_cols.append(perc_contrib_col_name)
    purchase_labels.append(label)
    purchase_ids_assets.append(f"{asset}_ID{purchase_id}") # Уникальный идентификатор для цвета

    historical_prices_filtered[value_col_name] = 0.0
    historical_prices_filtered[pnl_col_name] = 0.0
    historical_prices_filtered[perc_contrib_col_name] = 0.0

    mask = historical_prices_filtered.index >= purchase_time
    if mask.any():
        current_prices = historical_prices_filtered.loc[mask, price_col]
        if pd.isna(purchase_price) or purchase_price <= 0: # Check for non-positive price
            print(f"Предупреждение: Нулевая, отрицательная или NaN цена покупки для {asset} ID {purchase_id} в {purchase_time}. P&L будет 0.")
            price_ratio = pd.Series(0.0, index=current_prices.index)
        else:
            price_ratio = current_prices / purchase_price
            price_ratio = price_ratio.fillna(0).replace([np.inf, -np.inf], 0)

        current_purchase_value = initial_investment * price_ratio
        historical_prices_filtered.loc[mask, value_col_name] = current_purchase_value
        historical_prices_filtered.loc[mask, pnl_col_name] = current_purchase_value - initial_investment
    else:
        print(f"Предупреждение: Нет данных после времени покупки {purchase_time} для {asset} ID {purchase_id}.")

# Расчет общей стоимости и P&L
historical_prices_filtered['Total_Value_Relative'] = historical_prices_filtered[purchase_value_cols].sum(axis=1)
# Ensure Total_Value_Relative is not zero before calculating PnL relative to it
historical_prices_filtered['Total_PnL'] = historical_prices_filtered['Total_Value_Relative'] - historical_prices_filtered['Cumulative_Cost']

# Расчет процентного вклада P&L
print("Расчет процентного вклада P&L...")
for pnl_col, perc_contrib_col in zip(purchase_pnl_cols, purchase_perc_contrib_cols):
    # Avoid division by zero or near-zero total value
    denom = historical_prices_filtered['Total_Value_Relative']
    percentage_contribution = np.zeros_like(denom) # Initialize with zeros
    # Calculate only where denominator is significantly different from zero
    valid_denom_mask = np.abs(denom) > 1e-9
    percentage_contribution[valid_denom_mask] = (historical_prices_filtered.loc[valid_denom_mask, pnl_col] / denom[valid_denom_mask]) * 100
    historical_prices_filtered[perc_contrib_col] = pd.Series(percentage_contribution, index=historical_prices_filtered.index).fillna(0)


# Расчет общего P&L в процентах от общей стоимости
denom = historical_prices_filtered['Total_Value_Relative']
total_pnl_percentage = np.zeros_like(denom)
valid_denom_mask = np.abs(denom) > 1e-9
total_pnl_percentage[valid_denom_mask] = (historical_prices_filtered.loc[valid_denom_mask, 'Total_PnL'] / denom[valid_denom_mask]) * 100
historical_prices_filtered['Total_PnL_Percentage'] = pd.Series(total_pnl_percentage, index=historical_prices_filtered.index).fillna(0)


print("Расчеты завершены.")

# --- Шаг 4: Визуализация с Plotly ---

# Устанавливаем темную тему по умолчанию
pio.templates.default = "plotly_dark"

# Создаем фигуру с 3 подграфиками
fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.04, # Небольшое расстояние между графиками
    subplot_titles=(
        f'Стоимость портфеля vs Вложенные средства ({days_history} дней)',
        'Вклад каждой инвестиции в Абсолютную Прибыль/Убыток (P&L)',
        'Вклад P&L каждой инвестиции в % от Общей Стоимости Портфеля'
    )
)

# --- График 1: Общая стоимость и Кумулятивная стоимость ---
fig.add_trace(go.Scatter(
    x=historical_prices_filtered.index,
    y=historical_prices_filtered['Total_Value_Relative'],
    mode='lines',
    name='Общая стоимость',
    line=dict(color='#388BFF', width=2), # Яркий синий
    hovertemplate='Дата: %{x}<br>Стоимость: %{y:,.2f} USDT<extra></extra>'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=historical_prices_filtered.index,
    y=historical_prices_filtered['Cumulative_Cost'],
    mode='lines',
    name='Вложено средств',
    line=dict(color='#AAAAAA', dash='dash', width=1.5), # Серый пунктир
    hovertemplate='Дата: %{x}<br>Вложено: %{y:,.2f} USDT<extra></extra>'
), row=1, col=1)

# Отметки о покупках на первом графике
portfolio_in_range = portfolio_df[portfolio_df['Actual_Purchase_Time_Index'] >= start_date_history]
purchase_times = []
purchase_values = []
purchase_hover_texts = []

for index, row in portfolio_in_range.iterrows():
    plot_time = row['Actual_Purchase_Time_Index']
    if plot_time in historical_prices_filtered.index:
        value_at_purchase = historical_prices_filtered.loc[plot_time, 'Total_Value_Relative']
        purchase_times.append(plot_time)
        purchase_values.append(value_at_purchase)
        purchase_hover_texts.append(
            f"<b>Покупка {row['Актив']}</b><br>" +
            f"Сумма: +${row['Общая стоимость']:.2f}<br>" +
            f"Дата: {plot_time.strftime('%Y-%m-%d %H:%M')}<br>" +
            f"Цена актива: {row['Purchase_Price_Actual']:.4f}<br>" +
            f"Стоимость портф. в момент: {value_at_purchase:,.2f} USDT" +
            "<extra></extra>" # Убирает доп. инфо трейса
        )
    else:
         print(f"Предупреждение: Время покупки {plot_time} для {row['Актив']} ID {row['ID']} не найдено в индексе для графика 1.")

if purchase_times: # Добавляем трейс только если есть что добавить
    fig.add_trace(go.Scatter(
        x=purchase_times,
        y=purchase_values,
        mode='markers',
        name='Покупки',
        marker=dict(color='#FF5733', size=7, symbol='circle', line=dict(color='white', width=1)), # Оранжево-красный маркер
        hoverinfo='text', # Используем только кастомный текст
        text=purchase_hover_texts # Наш кастомный текст
    ), row=1, col=1)


# --- График 2: Вклад каждой покупки в Абсолютную Прибыль/Убыток (P&L) ---

# Генерируем цвета (используем Plotly Express для палитры)
num_colors = len(purchase_labels)
# colors = px.colors.qualitative.Plotly # Стандартная палитра
colors = px.colors.qualitative.T10 # Другая палитра
if num_colors > len(colors): # Если покупок больше, чем цветов, повторяем
    colors = colors * (num_colors // len(colors)) + colors[:num_colors % len(colors)]

# Создаем словарь цветов для консистентности
color_map = {label: colors[i] for i, label in enumerate(purchase_labels)}

# Используем stackgroup для P&L каждой покупки
for i, (pnl_col, label) in enumerate(zip(purchase_pnl_cols, purchase_labels)):
    color = color_map[label]
    fig.add_trace(go.Scatter(
        x=historical_prices_filtered.index,
        y=historical_prices_filtered[pnl_col].fillna(0), # Заполняем NaN нулями для stackplot
        mode='lines',
        name=label, # Используем полную метку для легенды
        stackgroup='pnl_absolute', # Группа для стекинга
        line=dict(width=0), # Убираем линию области
        fillcolor=color, # Цвет заливки
        hovertemplate=f'<b>{label}</b><br>Дата: %{{x}}<br>Абс. P&L: %{{y:,.2f}} USDT<extra></extra>',
        legendgroup=label # Группировка в легенде
        # showlegend=False # Можно скрыть отдельные элементы стека из легенды, если нужно
    ), row=2, col=1)

# Линия общего P&L для сравнения
fig.add_trace(go.Scatter(
    x=historical_prices_filtered.index,
    y=historical_prices_filtered['Total_PnL'],
    mode='lines',
    name='Общий P&L',
    line=dict(color='white', dash='dot', width=2), # Белая пунктирная линия
    hovertemplate='<b>Общий P&L</b><br>Дата: %{x}<br>P&L: %{y:,.2f} USDT<extra></extra>',
    legendgroup="total_pnl" # Отдельная группа
), row=2, col=1)

# Линия нуля (уровень безубыточности)
fig.add_hline(y=0, line_width=1, line_dash="solid", line_color="grey", row=2, col=1)


# --- График 3: Вклад P&L каждой покупки в % от Общей Стоимости Портфеля ---
for i, (perc_contrib_col, label) in enumerate(zip(purchase_perc_contrib_cols, purchase_labels)):
    color = color_map[label] # Используем тот же цвет
    fig.add_trace(go.Scatter(
        x=historical_prices_filtered.index,
        y=historical_prices_filtered[perc_contrib_col].fillna(0), # Заполняем NaN нулями
        mode='lines',
        name=label, # Полная метка
        stackgroup='pnl_percentage', # Другая группа стекинга
        line=dict(width=0),
        fillcolor=color,
        hovertemplate=f'<b>{label}</b><br>Дата: %{{x}}<br>% Вклад P&L: %{{y:.2f}}%<extra></extra>',
        legendgroup=label, # Та же группа легенды, что и в абс. P&L
        showlegend=False # Скрываем повторные метки в легенде
    ), row=3, col=1)

# Линия общего P&L в % от общей стоимости
fig.add_trace(go.Scatter(
    x=historical_prices_filtered.index,
    y=historical_prices_filtered['Total_PnL_Percentage'],
    mode='lines',
    name='Общий P&L %',
    line=dict(color='white', dash='dot', width=2),
    hovertemplate='<b>Общий P&L %</b><br>Дата: %{x}<br>P&L: %{y:.2f}%<extra></extra>',
    legendgroup="total_pnl_perc" # Отдельная группа
), row=3, col=1)

# Линия нуля
fig.add_hline(y=0, line_width=1, line_dash="solid", line_color="grey", row=3, col=1)

# --- Настройка макета ---
fig.update_layout(
    height=900, # Увеличиваем высоту для 3 графиков
    # title_text=f'Анализ портфеля за {days_history} дней ({start_date_history.date()} - {today.date()})', # Главный заголовок (можно убрать, т.к. есть заголовки подграфиков)
    hovermode='x unified', # Режим ховера, 'x unified' показывает все значения для данной x-координаты
    legend=dict(
        traceorder='normal', # Порядок в легенде как в коде
        orientation='h', # Горизонтальная легенда
        yanchor='bottom',
        y=1.02, # Размещение над верхним графиком
        xanchor='right',
        x=1
    )
)

# Обновление осей
fig.update_xaxes(
    showline=True, linewidth=1, linecolor='grey', mirror=True,
    gridcolor='rgba(128, 128, 128, 0.2)' # Полупрозрачная сетка
)
fig.update_yaxes(
    showline=True, linewidth=1, linecolor='grey', mirror=True,
    gridcolor='rgba(128, 128, 128, 0.2)',
    zeroline=False # Убираем явную нулевую линию, т.к. добавили через add_hline
)

# Настройка осей Y для каждого подграфика
fig.update_yaxes(title_text="Стоимость (USDT)", tickprefix="$", row=1, col=1)
fig.update_yaxes(title_text="Абс. P&L (USDT)", tickprefix="$", row=2, col=1)
fig.update_yaxes(title_text="% Вклад P&L", ticksuffix="%", row=3, col=1)

# Настройка оси X (только для нижнего графика)
fig.update_xaxes(title_text="Дата", row=3, col=1)

# Показать график
fig.show()


# Вывод данных для проверки (остается без изменений)
print("\nДанные портфеля с найденными ценами:")
print(portfolio_df[['ID', 'Дата', 'Актив', 'Общая стоимость', 'Purchase_Price_Actual', 'Actual_Purchase_Time_Index']])

print(f"\nРассчитанные данные портфеля (последние 5 записей):")
cols_to_show = ['Cumulative_Cost', 'Total_Value_Relative', 'Total_PnL', 'Total_PnL_Percentage']
if purchase_pnl_cols:
    cols_to_show.append(purchase_pnl_cols[0])
if purchase_perc_contrib_cols:
    cols_to_show.append(purchase_perc_contrib_cols[0])

cols_to_show = [col for col in cols_to_show if col in historical_prices_filtered.columns]
if cols_to_show:
    print(historical_prices_filtered[cols_to_show].tail())
else:
    print("Нет данных для отображения в итоговой таблице.")

Поиск цен на момент покупки...
Расчет кумулятивной стоимости...
Расчет вклада, P&L и % вклада для каждой покупки...
Расчет процентного вклада P&L...
Расчеты завершены.



Данные портфеля с найденными ценами:
   ID                Дата     Актив  Общая стоимость  Purchase_Price_Actual  \
0   3 2025-01-12 14:29:48  HBARUSDT           1000.0                0.28239   
1   2 2025-02-09 14:21:24   LTCUSDT             20.0              108.02000   
2   1 2025-03-05 14:21:17   BTCUSDT           1000.0            88344.25000   
3   0 2025-04-01 14:21:01   BNBUSDT           1000.0              615.26000   

  Actual_Purchase_Time_Index  
0        2025-01-12 15:00:00  
1        2025-02-09 15:00:00  
2        2025-03-05 15:00:00  
3        2025-04-01 15:00:00  

Рассчитанные данные портфеля (последние 5 записей):
                     Cumulative_Cost  Total_Value_Relative   Total_PnL  \
Open time                                                                
2025-04-12 03:00:00           3020.0           2492.588953 -527.411047   
2025-04-12 04:00:00           3020.0           2499.663974 -520.336026   
2025-04-12 05:00:00           3020.0           2503.473843 -51